In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib 

try:
    # --- 1. Carregamento e Preparação ---
    # AJUSTE 1: Corrigir o caminho do arquivo para o nome local.
    file_path = '/kaggle/input/jogos-completo/DB_completo (1).csv' 
    df = pd.read_csv(file_path, low_memory=False)
    print("--- Análise Inicial do Dataset ---")
    print(f"O dataset possui {df.shape[0]} linhas e {df.shape[1]} colunas.")

    df.dropna(subset=['preco_dolar'], inplace=True)
    for col in ['developers', 'publishers']:
        if col in df.columns:
            df[col] = df[col].fillna('Desconhecido')

    # --- Engenharia de Features de Tempo ---
    print("\n--- Criando features de tempo ---")
    df['release_year'] = df['release_year'].astype(int)
    df['release_month'] = df['release_month'].astype(int)
    df['release_date'] = pd.to_datetime(
        df['release_year'].astype(str) + '-' + df['release_month'].astype(str) + '-01',
        errors='coerce'
    )
    
    # AJUSTE 2: Reativar a remoção de datas nulas para evitar erros.
    df.dropna(subset=['release_date'], inplace=True)

    df['ano_lancamento'] = df['release_date'].dt.year
    df['mes_lancamento'] = df['release_date'].dt.month
    df['trimestre'] = df['release_date'].dt.quarter
    df['time_idx'] = (df['release_date'] - df['release_date'].min()).dt.days

    # --- 2. Pré-processamento ---
    print("\n--- Iniciando Pré-processamento ---")
    target = 'preco_dolar'

    print("\n--- Agrupando categorias raras ---")
    min_count_publisher = 20
    publishers_counts = df['publishers'].value_counts()
    rare_publishers = publishers_counts[publishers_counts < min_count_publisher].index
    df['publishers'] = df['publishers'].replace(rare_publishers, 'Outro')

    min_count_developer = 20
    developers_counts = df['developers'].value_counts()
    rare_developers = developers_counts[developers_counts < min_count_developer].index
    df['developers'] = df['developers'].replace(rare_developers, 'Outro')

    print("\n--- Criando features de popularidade ---")
    df['dev_popularity'] = df['developers'].map(df['developers'].value_counts())
    df['pub_popularity'] = df['publishers'].map(df['publishers'].value_counts())

    # Selecionar features e alvo
    features = df.drop(columns=[target, 'preco_euro', 'gameid', 'title', 'release_date', 'release_year', 'release_month'])
    
    # AJUSTE 3: Remover 'drop_first=True' para consistência com o dashboard.
    X_encoded = pd.get_dummies(features) 
    y = df[target]
    print(f"Dimensões de X após encoding: {X_encoded.shape}")

    # --- 3. Divisão dos Dados em Treino e Teste ---
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
    print(f"\nDados divididos: {len(X_train)} para treino, {len(X_test)} para teste.")

    # --- 4. Treinamento do Modelo ---
    print("\n--- Treinando o Modelo Random Forest Regressor ---")
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1) # Reduzido n_estimators para um treino mais rápido
    rf_model.fit(X_train, y_train)
    print("Modelo treinado com sucesso!")

    # --- 5. Avaliação do Modelo ---
    print("\n--- Avaliando a Performance do Modelo ---")
    y_pred = rf_model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Erro Médio Absoluto (MAE): ${mae:.2f}")
    print(f"R-squared (R²): {r2:.2f}")

    # --- 6. Importância das Variáveis (Opcional) ---
    # (Pode manter esta seção para sua análise)

    # --- 7. Salvar o Modelo e as Colunas ---
    print("\n--- Salvando o modelo e as colunas ---")
    joblib.dump(rf_model, 'modelo_regressao_preco.joblib')
    joblib.dump(X_encoded.columns, 'colunas_regressao_preco.joblib')
    print("Modelo de Regressão e colunas salvos com sucesso!")

except FileNotFoundError:
    print(f"ERRO: O arquivo '{file_path}' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

--- Análise Inicial do Dataset ---
O dataset possui 158742 linhas e 16 colunas.

--- Criando features de tempo ---

--- Iniciando Pré-processamento ---

--- Agrupando categorias raras ---

--- Criando features de popularidade ---
Dimensões de X após encoding: (158741, 1782)

Dados divididos: 126992 para treino, 31749 para teste.

--- Treinando o Modelo Random Forest Regressor ---
Modelo treinado com sucesso!

--- Avaliando a Performance do Modelo ---
Erro Médio Absoluto (MAE): $3.00
R-squared (R²): 0.61

--- Salvando o modelo e as colunas ---
Modelo de Regressão e colunas salvos com sucesso!
